# LSTM Regressor

Autoregression and predict one day ahead

The main idea to train and test the model is shown as follows:
1. Split the training set and test set
2. Use the cross validation method to find the optimal combination of hyperparameters
3. Use the model to predict the data
4. Data visualization

Import library needed in the model

In [21]:
from tensorflow.keras.layers import Dense,RepeatVector, LSTM, Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, Dropout
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Split the training set and test set

In [22]:
df=pd.read_csv('../dataset_for_analysis/data_weather_new1.csv',parse_dates=['date'])
df.set_index('date', inplace=True)

# Select the data used for model training
start_date='2000-01-01'
end_date='2020-12-31'
training_data=df[start_date:end_date]

X_mean_temp=training_data[['mean_temp','precipitation']]

In [23]:
X_mean_temp

,mean_temp,precipitation
date,,
2000-01-01,7.0,0.0
2000-01-02,7.9,0.2
2000-01-03,9.4,6.0
2000-01-04,7.0,0.2
2000-01-05,6.4,0.8
...,...,...
2020-12-27,7.5,2.0
2020-12-28,1.1,0.2
2020-12-29,2.6,0.0


Scaling the data

In [24]:
from sklearn.preprocessing import MinMaxScaler
scalar=MinMaxScaler(feature_range=(-1,1))
scaled_mean_temp=scalar.fit_transform(X_mean_temp[['mean_temp','precipitation']])

In [25]:
scaled_mean_temp

array([[-0.32110092, -1.        ],
       [-0.26605505, -0.98019802],
       [-0.17431193, -0.40594059],
       ...,
       [-0.59021407, -1.        ],
       [-0.58409786, -1.        ],
       [-0.79816514, -1.        ]])

In [26]:
timestep=30
X=[]
Y=[]
for i in range(len(X_mean_temp)-timestep):
    X.append(scaled_mean_temp[i:i+timestep])
    Y.append(scaled_mean_temp[i+timestep])

In [27]:
X=np.asanyarray(X)

In [28]:
Y=np.asanyarray(Y)

In [29]:
k1 = 6910
k2 = 7275
Xtrain = X[:k1,:,:]
Xtest = X[k1:k2,:,:]
X2020 = X[k2:,:,:]
Ytrain = Y[:k1]
Ytest= Y[k1:k2]
Y2020 = Y[k2:]

In [30]:
Xtrain.shape

(6910, 30, 2)

In [31]:
Xtest.shape

(365, 30, 2)

In [32]:
X2020.shape

(366, 30, 2)

## Single output (predict one day ahead)

Create the instance of tensorboard to visualize the results

In [34]:
from tensorflow.keras.callbacks import TensorBoard
tensorboard_callback = TensorBoard(log_dir="../lstm_results/mean_temp_and_precipitation_10echos")

In [ ]:
# Initiate the model
model = Sequential()

# Define the input and convolution layer
model.add(Conv1D(filters=256, kernel_size=2, activation='relu', input_shape=(30,2)))
model.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(RepeatVector(30))

# Define the LSTM layer
model.add(LSTM(units=100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=100, return_sequences=True))
model.add(LSTM(units=100, return_sequences=True))

# Define the bidirectional LSTM layer
model.add(Bidirectional(LSTM(128, activation='relu')))

# Define the dense and output layer
model.add(Dense(100, activation='relu'))
model.add(Dense(2))

# Compile the model
model.compile(loss='mse', optimizer='adam')
history = model.fit(Xtrain,Ytrain,validation_data=(Xtest,Ytest), epochs=10, verbose=1, callbacks=[tensorboard_callback])

Epoch 1/10
216/216 [==============================] - 46s 169ms/step - loss: 0.0709 - val_loss: 0.0634
Epoch 2/10
216/216 [==============================] - 31s 144ms/step - loss: 0.0592 - val_loss: 0.0594
Epoch 3/10
216/216 [==============================] - 32s 146ms/step - loss: 0.0571 - val_loss: 0.0589
Epoch 4/10
216/216 [==============================] - 27s 124ms/step - loss: 0.0574 - val_loss: 0.0578
Epoch 5/10
216/216 [==============================] - 30s 139ms/step - loss: 0.0556 - val_loss: 0.0594
Epoch 6/10
216/216 [==============================] - 29s 132ms/step - loss: 0.0545 - val_loss: 0.0554
Epoch 7/10
  6/216 [..............................] - ETA: 55s - loss: 0.0452

Save the model

In [ ]:
model.save('../lstm_results/lstm_mean_temp_and_precipitation.hdf5')

### Module evaluation and results visualizaiton

In [ ]:
from tensorflow.keras.models import load_model
model=load_model('../lstm_results/lstm_mean_temp_and_precipitation.hdf5')

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png')

In [ ]:
predicted_result=model.predict(X2020)

In [ ]:
predicted_result=scalar.inverse_transform(predicted_result)
actual_ytest=scalar.inverse_transform(Y2020)

In [ ]:
predicted_result

In [ ]:
actual_ytest

In [ ]:
from sklearn.metrics import mean_squared_error
print(f'The MSE of mean_temp prediction: {mean_squared_error(predicted_result[:,0],actual_ytest[:,0]):.3f}')
print(f'The MSE of precipitation prediction: {mean_squared_error(predicted_result[:,1],actual_ytest[:,1]):.3f}')

Mean_temp

In [ ]:
plt.figure(figsize=(20,9))
plt.plot(actual_ytest[:,0], 'blue')
plt.plot(predicted_result[:,0],'r')
plt.legend(('Test','Predicted'))
plt.show()

Precipitation

In [ ]:
plt.figure(figsize=(20,9))
plt.plot(actual_ytest[:,1], 'blue')
plt.plot(predicted_result[:,1],'r')
plt.legend(('Test','Predicted'))
plt.show()

## Multiple output (predict one week ahead)